In [1]:
import Qubos as Q
from IPython.display import Math

## Working with States

Qubos defines two main classes: `State` and `Map`. We'll start by looking at the `State` class and what it can do. Let's first define a `State` object consisting of 3 qubits, all in the ground state (i.e. the state $|000\rangle$).

In [2]:
my_qubits = Q.State(3)
my_qubits

If you want to visualize the state, you can use the `.to_latex()` function:

In [3]:
Q.to_latex(my_qubits)

<IPython.core.display.Math object>

If you forget how many qubits are in the state, you can use the `.n()` method:

In [4]:
my_qubits.n()

3

One can also apply any Clifford unitaries to the state:
* `apply_h(state, pos)` applies a Hadamard to the qubit at position `pos`
* `apply_cx(state, pos0, pos1)` applies a CNOT with the qubit at position `pos0` as control and the qubit at position `pos1` as target
* `apply_s(state, pos)` applies an S gate to the qubit at position `pos`
* `apply_swap(state, pos0, pos1)` applies a SWAP gate with the qubit at position `pos0` and the qubit at position `pos1`
* `apply_x(state, pos)` applies an X gate to the qubit at position `pos`
* `apply_z(state, pos)` applies a Z gate to the qubit at position `pos`


In [5]:
Q.apply_h(my_qubits, 0)     # applies a Hadamard to the first qubit
Q.apply_cx(my_qubits, 0, 1) # applies a CNOT with the first qubit as control and the second as target
Q.apply_cx(my_qubits, 1, 2) # applies a CNOT with the second qubit as control and the third as target
Q.apply_s(my_qubits, 0)     # applies an S gate to the first qubit
Q.to_latex(my_qubits)

<IPython.core.display.Math object>

Checking equality of states is as simple as:

In [6]:
state0 = Q.State(2)
Q.apply_h(state0, 0)
Q.apply_cx(state0, 0, 1)


state1 = Q.State(2)
Q.apply_h(state1, 1)
Q.apply_cx(state1, 1, 0)

[state0 == state1, state0 == my_qubits]

[True, False]

Part of the innovation of Qubos is that one can also append qubits and postselect on them while still accurately keeping track of the global phase. For example, the following code starts with a scalar state, appends a qubit, applies a Hadamard and then postselects on the first qubit being in the state $|0\rangle$ - i.e. it computes the operation $\langle 0 | H | 0 \rangle$.

In [7]:
my_qubits = Q.State()
my_qubits.zero_prep()
Q.apply_h(my_qubits, 0)
my_qubits.zero_postselect(0) # postselects on the first qubit (on position 0) being in the state $|0\rangle$

Q.to_latex(my_qubits)

<IPython.core.display.Math object>

One can insert new qubits or postselect at arbitrary positions:

In [8]:
my_qubits = Q.State()
for i in range(3):
    my_qubits.zero_prep(0)
    Q.apply_x(my_qubits, 0)

display(Q.to_latex(my_qubits)) # |111>

my_qubits.zero_prep(1)
display(Q.to_latex(my_qubits)) # |1011>

my_qubits.zero_postselect(1)
display(Q.to_latex(my_qubits)) # |111>

my_qubits.zero_postselect(1)
display(Q.to_latex(my_qubits)) # 0

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

## The Insides of a State

Any non-zero stabilizer state $| \psi \rangle$ has the property that it can be written as follows:

$$
| \psi \rangle = e^\frac{2 p i \pi}{8} \frac{1}{\sqrt{2^m}} \sum_{A \vec{x} = \vec{b}} i^{\vec{x} \cdot \ell + \vec{x}^\intercal Q \vec{x} } | x \rangle
$$

where:
* $A$ and $b$ are a boolean matrix and a vector of integers of size $n \times m$ and $n$ respectively, defining a system of linear equations $A \vec{x} = \vec{b}$
* $\ell$ is a vector of integers of size $m$
* $Q$ is a boolean matrix of size $m \times m$
* $p$ is an integer in the interval $\{0, \dots, 7\}$
* $m$ is an integer

All of these correspond to the following methods of the `State` class:
* `.is_zero()` returns a boolean corresponding to whether the state is the zero state $ $
* `.magnitude()` corresponds to $m$
* `.phase()` corresponds to $p$
* `.affine_part()` returns a tuple with the numpy matrices corresponding to $A$ and $b$
* `.phase_polynomial_matrix()` returns a numpy matrix corresponding to $Q$
* `.lin_part()` returns a numpy vector corresponding to $\ell$


## Working with Maps